## بخش اول تمرین 1

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.Preprocessing import StandardScaler, Label Encoder, and OneHotEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# 1. وارد کردن داده‌ها
df = pd.read_csv('loans.csv')
print(df)

     client_id loan_type  loan_amount  repaid  loan_id  loan_start  \
0        46109      home        13672       0    10243  2002-04-16   
1        46109    credit         9794       0    10984  2003-10-21   
2        46109      home        12734       1    10990  2006-02-01   
3        46109      cash        12518       1    10596  2010-12-08   
4        46109    credit        14049       1    11415  2010-07-07   
..         ...       ...          ...     ...      ...         ...   
438      26945     other        12963       0    10330  2001-11-26   
439      26945    credit         1728       1    10248  2004-01-27   
440      26945     other         9329       0    10154  2001-12-17   
441      26945      home         4197       0    10333  2003-10-16   
442      26945      home         3643       0    11434  2010-03-24   

       loan_end  rate  
0    2003-12-20  2.15  
1    2005-07-17  1.25  
2    2007-07-05  0.68  
3    2013-05-05  1.24  
4    2012-05-21  3.13  
..          ...

In [5]:
# 2. بررسی اولیه داده‌ها
print("اطلاعات کلی دیتاست:")
print(df.info())
print("نمونه‌ای از داده‌ها:")
print(df.head())


اطلاعات کلی دیتاست:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client_id    443 non-null    int64  
 1   loan_type    443 non-null    object 
 2   loan_amount  443 non-null    int64  
 3   repaid       443 non-null    int64  
 4   loan_id      443 non-null    int64  
 5   loan_start   443 non-null    object 
 6   loan_end     443 non-null    object 
 7   rate         443 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 27.8+ KB
None
نمونه‌ای از داده‌ها:
   client_id loan_type  loan_amount  repaid  loan_id  loan_start    loan_end  \
0      46109      home        13672       0    10243  2002-04-16  2003-12-20   
1      46109    credit         9794       0    10984  2003-10-21  2005-07-17   
2      46109      home        12734       1    10990  2006-02-01  2007-07-05   
3      46109      cash        12518       1    

In [6]:
# 3. مدیریت داده‌های گمشده
imputer = SimpleImputer(strategy='median')
df[df.select_dtypes(include=[np.number]).columns] = imputer.fit_transform(df.select_dtypes(include=[np.number]))

df.fillna("Unknown", inplace=True)  # برای متغیرهای غیر عددی

In [7]:
# 4. تشخیص داده‌های پرت و پردازش آن‌ها
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])


In [8]:
# 5. تبدیل متغیرهای عددی
for col in numeric_cols:
    df[col] = np.log1p(df[col])  # تبدیل لگاریتمی برای کاهش نوسانات


In [9]:
# 6. استانداردسازی متغیرهای عددی
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [10]:
# 7. کدگذاری متغیرهای دسته‌ای
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].nunique() <= 10:
        df = pd.get_dummies(df, columns=[col], drop_first=True)  # OneHot Encoding
    else:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

In [14]:
print(df.columns)  # نمایش نام تمام ستون‌های موجود
# 8. تقسیم داده‌ها به آموزش و آزمایش

# تقسیم داده‌ها به متغیرهای مستقل (X) و وابسته (y)
X = df.drop(columns=['repaid'])  # حذف ستون هدف از ویژگی‌ها
y = df['repaid']  # متغیر هدف

# تقسیم داده‌ها به مجموعه‌های آموزشی و آزمایشی
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ذخیره داده‌های پردازش‌شده
X_train.to_csv('X_train_processed.csv', index=False)
X_test.to_csv('X_test_processed.csv', index=False)
y_train.to_csv('y_train_processed.csv', index=False)
y_test.to_csv('y_test_processed.csv', index=False)

print("پیش‌پردازش داده‌ها تکمیل شد و داده‌های پردازش‌شده ذخیره شدند.")


پیش‌پردازش داده‌ها تکمیل شد و داده‌های پردازش‌شده ذخیره شدند.


## بخش دوم تمرین اول

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# خواندن داده‌ها
df = pd.read_csv("loans.csv")

# نمایش اطلاعات اولیه دیتافریم
print(df.info())

# انتخاب متغیرهای مستقل و متغیر هدف
X = df.drop(columns=['loan_amount'])  # همه ویژگی‌ها به جز loan_amount
y = df['loan_amount']  # مقدار وام به عنوان متغیر هدف

# تقسیم داده‌ها به مجموعه آموزشی و آزمایشی (۸۰٪ آموزش، ۲۰٪ تست)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client_id    443 non-null    int64  
 1   loan_type    443 non-null    object 
 2   loan_amount  443 non-null    int64  
 3   repaid       443 non-null    int64  
 4   loan_id      443 non-null    int64  
 5   loan_start   443 non-null    object 
 6   loan_end     443 non-null    object 
 7   rate         443 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 27.8+ KB
None


In [16]:
# انتخاب یک ویژگی (مثلاً نرخ بهره 'rate')
X_train_single = X_train[['rate']]
X_test_single = X_test[['rate']]

# مدل‌سازی
model_single = LinearRegression()
model_single.fit(X_train_single, y_train)

# پیش‌بینی مقادیر
y_pred_single = model_single.predict(X_test_single)

# ارزیابی مدل
mse_single = mean_squared_error(y_test, y_pred_single)
r2_single = r2_score(y_test, y_pred_single)

print(f"نتایج مدل رگرسیون خطی ساده:")
print(f"Mean Squared Error: {mse_single}")
print(f"R² Score: {r2_single}")


نتایج مدل رگرسیون خطی ساده:
Mean Squared Error: 15486298.545382299
R² Score: -0.011749047739805007


In [21]:
print(df.dtypes)


client_id        int64
loan_type       object
loan_amount      int64
repaid           int64
loan_id          int64
loan_start      object
loan_end        object
rate           float64
dtype: object


In [22]:
# پیدا کردن همه ستون‌های متنی (دسته‌بندی‌شده)
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
print("ستون‌های دسته‌بندی‌شده:", categorical_columns)


ستون‌های دسته‌بندی‌شده: ['loan_type', 'loan_start', 'loan_end']


In [23]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# پیدا کردن تنها ستون دسته‌ای (loan_type)
categorical_columns = ['loan_type']

# اجرای One-Hot Encoding
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))

# حذف ستون اصلی و اضافه کردن ستون‌های جدید
df = df.drop(columns=categorical_columns)
df = pd.concat([df, X_encoded], axis=1)

print("داده‌های تبدیل‌شده:", df.head())


داده‌های تبدیل‌شده:    client_id  loan_amount  repaid  loan_id  loan_start    loan_end  rate  \
0      46109        13672       0    10243  2002-04-16  2003-12-20  2.15   
1      46109         9794       0    10984  2003-10-21  2005-07-17  1.25   
2      46109        12734       1    10990  2006-02-01  2007-07-05  0.68   
3      46109        12518       1    10596  2010-12-08  2013-05-05  1.24   
4      46109        14049       1    11415  2010-07-07  2012-05-21  3.13   

   loan_type_credit  loan_type_home  loan_type_other  
0               0.0             1.0              0.0  
1               1.0             0.0              0.0  
2               0.0             1.0              0.0  
3               0.0             0.0              0.0  
4               1.0             0.0              0.0  


In [ ]:
import pandas as pd

# تبدیل ستون‌های تاریخ به فرمت datetime
df["loan_start"] = pd.to_datetime(df["loan_start"])
df["loan_end"] = pd.to_datetime(df["loan_end"])

# تبدیل تاریخ‌ها به تعداد روز از تاریخ مرجع (مثلاً ۱ ژانویه ۲۰۰0)
reference_date = pd.Timestamp("2000-01-01")
df["loan_start_days"] = (df["loan_start"] - reference_date).dt.days
df["loan_end_days"] = (df["loan_end"] - reference_date).dt.days

# حذف ستون‌های متنی تاریخ
df = df.drop(columns=["loan_start", "loan_end"])


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# انتخاب ویژگی‌ها (همه ستون‌ها به جز ستون هدف)
X = df.drop(columns=["repaid"])  # فرض بر اینکه ستون هدف 'repaid' است
y = df["repaid"]

# تقسیم داده‌ها به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# مدل رگرسیون خطی چندمتغیره
model_multi = LinearRegression()
model_multi.fit(X_train, y_train)

# پیش‌بینی
y_pred_multi = model_multi.predict(X_test)

# ارزیابی مدل
mse_multi = mean_squared_error(y_test, y_pred_multi)
r2_multi = r2_score(y_test, y_pred_multi)

print(f"\nنتایج مدل رگرسیون خطی چندمتغیره:")
print(f"Mean Squared Error: {mse_multi}")
print(f"R² Score: {r2_multi}")



نتایج مدل رگرسیون خطی چندمتغیره:
Mean Squared Error: 0.28096662918603277
R² Score: -0.25736535015964157
